In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import rdkit 
import multiprocessing
import copy
import math 
import random
import pickle 
import utils 
import model 
from utils import parallel_f, get_mol, replace_atom, tokenize, pad, MyDataset, get_dic
from model import TransformerVAE, device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   

In [3]:
with open('data/chembl24_canon_train.pickle', 'rb') as file :
    smi_list = pickle.load(file) 

random.shuffle(smi_list)
smi_list = smi_list[:200000]

In [4]:

mol_list = parallel_f(get_mol, smi_list)

smi_list = parallel_f(replace_atom, smi_list)
smi_dic = get_dic(smi_list)
inv_dic = {v:k for k, v in smi_dic.items()}
token_list = parallel_f(tokenize, smi_list)

max_len = len(max(token_list, key=len))
token_list = parallel_f(pad, token_list)


In [5]:
BATCH_SIZE = 128
dataset = MyDataset(token_list)

train_set, val_set = random_split(dataset, [0.95, 0.05])

train_loader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_set, batch_size = BATCH_SIZE, shuffle = True)

In [22]:
model = TransformerVAE(1024, 128, 8, 0.5, len(smi_dic)).to(device)

loss_fn = nn.NLLLoss()
optim = torch.optim.Adam(model.parameters(), lr = 0.0001) 

NUM_EPOCH = 1

In [23]:
for epoch in range(1, NUM_EPOCH + 1) :
    train_loss = 0
    val_loss = 0
    model.train()
    for input in train_loader :
        input = input.to(device)
        output = model(input, input[:, :-1])

        loss = loss_fn(output.reshape(-1, len(smi_dic)), input[:, 1:].reshape(-1)) + model.first_encoder.kl
        train_loss += loss.item()
        loss.backward()
        optim.step()
        optim.zero_grad()

    # model.eval()
    # for input in val_loader :
    #     input = input.to(device)
    #     output = model(input, input[:, :-1])
    #     loss = loss_fn(output.reshape(-1, len(smi_dic)), input[:, 1:].reshape(-1)) + model.first_encoder.kl
    #     val_loss += loss.item()
    print(f'epoch : {epoch}, train loss : {train_loss / len(train_loader)} val loss : {val_loss / len(val_loader)}')


epoch : 1, train loss : 627881.4107218013 val loss : 0.0


In [27]:
model.eval()
encoder = model.second_encoder
decoder = model.decoder


for _ in range(30) :
    z = torch.randn(1, max_len, 128).to(device)
    target = torch.zeros(1, 1, dtype=torch.long).to(device)

    for i in range(max_len - 1) :
        memory = encoder(z) 
        out = decoder(memory, target)
        _, idx = torch.topk(out, 1, dim = -1)
        idx = idx[:, -1, :]
        target = torch.cat([target, idx], dim = 1)

    target = target.squeeze(0).tolist()
    smiles = ''.join([inv_dic[i] for i in target])
    print(smiles)



<START>CCC(C(=CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCC(=O)NC(=O)NC(=O)NC(C)C(C)C)cc1)cccc1)C(=O)NCCC(C(=O)NCCCCC(CCC)C)C)CCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCC(=O)NC(=O)NC(=O)NC(C)C)cc1)C(=O)NC(CC)C)C)cc1ccc1<END><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD

<START>CCC(=O)cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
<START>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCC(=O)NC(=O)NC(=O)NC(C)C(C)C)cc1)cccc1)C(=O)NCCCC(CCC)C)CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCC(F)c1cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
<START>CCC(C(=CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
<START>CCCCCCCCCCCC

In [88]:
a = get_mol('C(=O)NC(=O)NC(=O)NC(=O)NC(=O)NC(=O)N')